<p><img style="padding: 0 15px; float: left;" src="Images/FTLogo300.jpg" alt="FT Crusader Logo" Width='140' Height= '250'/></p> 
<p> <h2> Frogtown DataLoad Workbook, 05/06/18</h2> <a name="tc"></a>
<p> <h3> By Frogtown Crusader (Abu Nayeem)</h3>

This is just clean coded process going over the uploading steps assuming all the preliminary steps after decoder are made


## Table of contents <a name="content"></a>
* [Data Setup](#setup)
* [Create Variables](#create)
* [Intersection Table: Preparation](#intersection)
* [Address Table: Preparation](#address)
* [Preliminary Data Cleaning](#pre)


### Data Setup <a name="setup"></a>

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline 
from IPython.display import HTML
from IPython.display import display
import requests # library to handle requests

#Load Data
#df_crime = pd.read_csv('Datasets/Crime_Incident_Report_-_Dataset.csv')

#rename columns
#cols= ['Case','Date','Time','Code','IncType','Incident','Grid','NNum','Neighborhood','Block','CallDispCode','CallDisposition', 'Count']
#df_crime.columns= cols

#selection for Frogtown and nearby area
#df=df_crime.query('Grid in [66,67, 68, 86, 87,88,89, 90, 91, 92,106,107,108,109,110]')

#Socarata upload Method
from sodapy import Socrata

#New Upload Method Get Information from Socrata API
client = Socrata("information.stpaul.gov", None)

#Easier to bulk upload
results = client.get("gppb-g9cg", limit=1000000)
df = pd.DataFrame.from_records(results)
# Find Max Date Value
results = client.get("gppb-g9cg", limit=1)
r_max = pd.DataFrame.from_records(results)

#rename columns [Note the order of Columns have changed]
cols= ['Block','CallDispCode','CallDisposition','Case','Code', 'Count','Date','Grid','Incident','IncType','Neighborhood','NNum','Time']
df.columns= cols
df=df.dropna()
df = df.astype({"Case": int, "Code": int, "Grid":float, "NNum":int,"Count":int})
#select respective Grids of interest
df=df.query("Grid in (66,67, 68, 86, 87,88,89, 90, 91, 92,106,107,108,109,110)")

df.head(4)

,Block,CallDispCode,CallDisposition,Case,Code,Count,Date,Grid,Incident,IncType,Neighborhood,NNum,Time
1,VANBUREN AV & COMO,A,Advised,21165906,2619,1,2021-08-11T23:47:34.000,90.0,Discharge,"Weapons, Discharging a Firearm in the City Limits",7 - Thomas/Dale(Frogtown),7,2021-08-11T23:47:34.000
2,THOMAS AV & GROTTO,A,Advised,21165903,9954,1,2021-08-11T23:46:19.000,88.0,Proactive Police Visit,Proactive Police Visit,7 - Thomas/Dale(Frogtown),7,2021-08-11T23:46:19.000
5,15X CHARLES AV,A,Advised,21165885,1800,1,2021-08-11T23:13:25.000,90.0,Narcotics,Narcotics,7 - Thomas/Dale(Frogtown),7,2021-08-11T23:13:25.000
6,55X CENTRAL AV W,A,Advised,21165880,9954,1,2021-08-11T23:01:57.000,109.0,Proactive Police Visit,Proactive Police Visit,8 - Summit/University,8,2021-08-11T23:01:57.000


### Create New Variables <a name="create"></a>

In [2]:
#Add Time Variables
df= df[df.Case != 18254093] #messed up time variable

#Convert Date to Datetime!
from datetime import datetime

df['DateTime']= pd.to_datetime(df['Date']) # Create new column called DateTime
df['Year']= df['DateTime'].dt.year #create year column
df['DayofWeek']=df['DateTime'].dt.dayofweek #create day of the week column where default 0=Monday
df['Weekend'] = df['DayofWeek'].apply(lambda x: 1 if (x>4)  else 0) #Create a weekend category
df['Month'] = df['DateTime'].dt.month # Create Month Category
df['Day'] = df['DateTime'].dt.day #Create Day of the Current month
df['DayYear'] = df['DateTime'].dt.dayofyear  #Create Day of the year (0-365)
df['Day_Max'] = df.iloc[0,-1] #selects uptodate day; NOTE: the data is sorted chronologically

#Hour Data
df['TimeHour']= pd.to_datetime(df['Time'])
df['Hour'] = df['TimeHour'].dt.hour.astype(int) #Create Hour Colum
df['LateNight'] = df['Hour'].apply(lambda x: 1 if (x>21 or x<5)  else 0) #Latenight designation from 10Pm to 6PM

#Creating the intersection Column. Note: the Block column has the address information
df.Block = df.Block.astype(str) #first change the type to string
df['Block']= df['Block'].str.lower() #lowercase string to create uniformity

#While scanning the data I noticed that all intersections had "&" 
df['Intersection'] = df['Block'].apply(lambda x: 1 if '&' in x else 0) #intersection

df.head(5)

,Block,CallDispCode,CallDisposition,Case,Code,Count,Date,Grid,Incident,IncType,...,DayofWeek,Weekend,Month,Day,DayYear,Day_Max,TimeHour,Hour,LateNight,Intersection
1,vanburen av & como,A,Advised,21165906,2619,1,2021-08-11T23:47:34.000,90.0,Discharge,"Weapons, Discharging a Firearm in the City Limits",...,2,0,8,11,223,223,2021-08-11 23:47:34,23,1,1
2,thomas av & grotto,A,Advised,21165903,9954,1,2021-08-11T23:46:19.000,88.0,Proactive Police Visit,Proactive Police Visit,...,2,0,8,11,223,223,2021-08-11 23:46:19,23,1,1
5,15x charles av,A,Advised,21165885,1800,1,2021-08-11T23:13:25.000,90.0,Narcotics,Narcotics,...,2,0,8,11,223,223,2021-08-11 23:13:25,23,1,0
6,55x central av w,A,Advised,21165880,9954,1,2021-08-11T23:01:57.000,109.0,Proactive Police Visit,Proactive Police Visit,...,2,0,8,11,223,223,2021-08-11 23:01:57,23,1,0
12,64x central av w,RR,Report Written,21165862,661,1,2021-08-11T21:54:00.000,108.0,Theft,"Theft, Bicycle Theft, Under $500",...,2,0,8,11,223,223,2021-08-11 21:54:00,21,0,0


### Prepare Intersection table <a name="intersection"></a>

In [3]:
#Load clean intersections key
df_Intersection_key = pd.read_csv('Datasets/Intersection_key_clean.csv')

# Create a new dateframe specifying only intersections from primary dataset
dfI=df.query('Intersection ==1')
print('The intersection table dimension are ' + str(dfI.shape))
#print(dfI.Block.head(10))


#Split the strings
new=dfI['Block'].str.split("& ", n = 1, expand = True) 
dfI['Inter2']= new[1]
new=dfI['Block'].str.split(" ", n = 1, expand = True) #Note the code specifies the first time a space occured
dfI['Inter1']=new[0]

#Create the IndexKey; recall we prepared the IntersectionKey where it considers any order
dfI['IndexKey']= dfI['Inter1']+ '_' + dfI['Inter2']
dfI.reset_index()
dfI=pd.merge(dfI, df_Intersection_key, on='IndexKey', how='left')



The intersection table dimension are (13128, 25)


In [5]:
#Drop rows with missing coordinates
dfI=dfI[dfI['Coordinates'].notnull()]

# Separate Latitude and Longitude 
new=dfI['Coordinates'].str.split(",", n = 1, expand = True) 
# making seperate first name column from new data frame 
dfI['Latitude']= pd.to_numeric(new[0]) #pd.to_numeric convert it to float
dfI['Longitude']= pd.to_numeric(new[1])

#Renaming columns

dfI['Block']=dfI['OutputKey'] #for practical purposes it makes sense
Drop_col=['Inter2','Inter1', 'IndexKey', 'Coordinates', 'OutputKey']
dfI_Final=dfI.drop(Drop_col, axis=1,)
dfI_Final.head(5)


,Block,CallDispCode,CallDisposition,Case,Code,Count,Date,Grid,Incident,IncType,...,Month,Day,DayYear,Day_Max,TimeHour,Hour,LateNight,Intersection,Latitude,Longitude
1,grotto_thomas,A,Advised,21165903,9954,1,2021-08-11T23:46:19.000,88.0,Proactive Police Visit,Proactive Police Visit,...,8,11,223,223,2021-08-11 23:46:19,23,1,1,44.959356,-93.131431
3,rice_sherburne,A,Advised,21165585,9954,1,2021-08-11T17:03:05.000,91.0,Proactive Police Visit,Proactive Police Visit,...,8,11,223,223,2021-08-11 17:03:05,17,0,1,44.956848,-93.105964
4,hamline_university,RR,Report Written,21165556,1820,1,2021-08-11T16:43:00.000,86.0,Narcotics,"Narcotics, Possession of Synthetic Narcotic, D...",...,8,11,223,223,2021-08-11 16:43:00,16,0,1,44.955811,-93.156832
5,rice_sherburne,A,Advised,21165552,9954,1,2021-08-11T16:39:05.000,91.0,Proactive Police Visit,Proactive Police Visit,...,8,11,223,223,2021-08-11 16:39:05,16,0,1,44.956848,-93.105964
6,jackson_university,RR,Report Written,21165594,710,1,2021-08-11T15:41:00.000,92.0,Auto Theft,"Motor Vehicle Theft, Automobile",...,8,11,223,223,2021-08-11 15:41:00,15,0,1,44.955956,-93.096982


### Prepare Address Decoder Table <a name="address"></a>

It is so remarkably short

In [5]:
#Split Data
dfW=df.query('Intersection==0')

#Load Complete Decoder Key
df_C= pd.read_csv('Datasets/SemiKey.csv')
df_C= df_C[['Block','Latitude','Longitude']]

# Merge with the dataset and remove missing values
dC=pd.merge(dfW, df_C, on='Block', how='left')
dC=dC.fillna('Mi')
dC=dC.query('Latitude != "Mi"')

#Bringing the data back together 
fg= dfI.append(dC, ignore_index=True)

### Preliminary Pre-Cleaning steps <a name="clean"></a>

It is good to do the data pre-cleaning steps here to reduce clutter on a working notebook. A few edits include renaming some values, clustering crimes together and creating some dummies. It is saved in a csv, which is used for execution

In [6]:
#Few Quick Edits
fg.CallDisposition.loc[(fg['CallDisposition'] == 'G - Gone on Arrival')] = 'Gone on Arrival'
fg.CallDisposition.loc[(fg['CallDisposition'] == 'A - Advised')] = 'Advised'
fg.CallDisposition.loc[(fg['CallDisposition'] == 'RR - Report Written')] = 'Report Written'
fg.Incident.loc[(fg['Incident'] == 'Simple Asasult Dom.')] = 'Simple Assault Dom.'
fg.Incident.loc[(fg['Incident'] == 'Graffiti')] = 'Vandalism'
fg.Incident.loc[fg["Incident"].isin([ "Rape","Agg. Assault",'Homicide'])]= 'Violent'
fg.Incident.loc[fg["Incident"].isin(["Simple Assault Dom.","Agg. Assault Dom."])]= 'Domestic Assault'


#[fg["Incident"].isin(["Simple Assault Dom.", "Rape"])


#Create a dummy for each crime category
fg= pd.concat([fg,pd.get_dummies(fg['Incident'])], axis=1)
fg= pd.concat([fg,pd.get_dummies(fg['CallDisposition'])], axis=1)

fg.to_csv('Datasets/FGCrime_Final.csv', encoding='utf-8', index=False)

In [8]:
fg.query('Year==2021')

,Block,CallDispCode,CallDisposition,Case,Code,Coordinates,Count,Date,DateTime,Day,...,Vandalism,Violent,10 - Como,11 - Hamline/Midway,13 - Union Park,7 - Thomas/Dale(Frogtown),8 - Summit/University,Advised,Gone on Arrival,Report Written
0,grotto_thomas,A,Advised,21165903,9954,"44.959356, -93.131431",1,2021-08-11T23:46:19.000,2021-08-11 23:46:19,11,...,0,0,0,0,0,0,0,1,0,0
1,rice_sherburne,A,Advised,21165585,9954,"44.956848, -93.105964",1,2021-08-11T17:03:05.000,2021-08-11 17:03:05,11,...,0,0,0,0,0,0,0,1,0,0
2,hamline_university,RR,Report Written,21165556,1820,"44.955811, -93.156832",1,2021-08-11T16:43:00.000,2021-08-11 16:43:00,11,...,0,0,0,0,0,0,0,0,0,1
3,rice_sherburne,A,Advised,21165552,9954,"44.956848, -93.105964",1,2021-08-11T16:39:05.000,2021-08-11 16:39:05,11,...,0,0,0,0,0,0,0,1,0,0
4,jackson_university,RR,Report Written,21165594,710,"44.955956, -93.096982",1,2021-08-11T15:41:00.000,2021-08-11 15:41:00,11,...,0,0,0,0,0,0,0,0,0,1
5,dale_thomas,A,Advised,21165391,9954,"44.959342, -93.126371",1,2021-08-11T12:57:32.000,2021-08-11 12:57:32,11,...,0,0,0,0,0,0,0,1,0,0
6,rice_university,A,Advised,21165325,9954,"44.955847, -93.105959",1,2021-08-11T11:20:31.000,2021-08-11 11:20:31,11,...,0,0,0,0,0,0,0,1,0,0
7,linden_mtairy,A,Advised,21165189,9954,"44.959025, -93.095111",1,2021-08-11T08:13:58.000,2021-08-11 08:13:58,11,...,0,0,0,0,0,0,0,1,0,0
8,rice_sherburne,A,Advised,21164616,9954,"44.956848, -93.105964",1,2021-08-10T14:01:49.000,2021-08-10 14:01:49,10,...,0,0,0,0,0,0,0,1,0,0
9,rice_charles,A,Advised,21164500,9954,"44.957614, -93.105951",1,2021-08-10T10:53:19.000,2021-08-10 10:53:19,10,...,0,0,0,0,0,0,0,1,0,0


In [7]:
Ward1.query('ElectionDescription in ("11/08/2016 - STATE GENERAL", "08/09/2016 - STATE PRIMARY","11/07/2017 - MUNICIPAL GENERAL","11/07/2017 - SCHOOL DISTRICT GENERAL"  ,"11/08/2016 - STATE GENERAL","08/09/2016 - STATE PRIMARY","11/03/2015 - MUNICIPAL GENERAL","11/03/2015 - SCHOOL DISTRICT GENERAL","11/08/2011 - MUNICIPAL GENERAL","11/06/2018 - STATE GENERAL","08/14/2018 - STATE PRIMARY")' 

SyntaxError: unexpected EOF while parsing (<ipython-input-7-b3cfb0eeffb8>, line 1)